# Naive Bayes

A probabilistic classifier.
Given $n$ _attributes_, that is

$$
\text{attributes} = \lbrace{a_1, a_2, a_3, \dots, a_n \rbrace},
$$

and a set of $k$ _labels_, we can calculate the conditional probability of each label given a certain set of attributes. That is:

$$
p\left(L_k|a_1, a_2, a_3,\dots,a_n\right)
$$.

This is done by direct application of Baye's theorem:

$$
p\left(L_k|a_1,a_2,a_3,\dots,a_n\right) = \frac{p\left(L_k\right)p\left(a_1,a_2,a_3,\dots,a_n|L_k\right)}{p\left(a_1,a_2,a_3,\dots,a_n\right)}
$$

## Example

For the following minimal example a Java implementation of the Naive Bayes algorithm will be used (available at https://github.com/ruivieira/java-naive-bayes).

Consider a simple IT purchasing system where the user can choose a laptop brand (`Apple` or `Lenovo`) for use in a specific department (`design` or `accounting`) in one of two offices (`US` or `UK`).

In this case, NB will try to classify the laptop brand according to the historical purchase data. It is clear then that the attributes will be:

$$
    a = \lbrace\text{user}, \text{department}, \text{office}\rbrace
$$

and the labels will be

$$
    L = \lbrace\text{Brand A}, \text{Brand B}\rbrace
$$

In [2]:
%maven org.ruivieira:naivebayes:0.1-SNAPSHOT

In [3]:
import org.ruivieira.ml.naivebayes.NaiveBayes;
import org.ruivieira.ml.naivebayes.Model;

import java.util.Map;

In [4]:
Model model = Model.create();

Let's start by adding the first record. User Anna buys an Apple for the US design department.

In [5]:
model.train(new String[]{"Anna", "design", "US"}, "Apple");

If we try to predict that label (outcome) for any of the attributes, the result will be unsurprising:

In [6]:
NaiveBayes naiveBayes = new NaiveBayes(model);
System.out.println("Anna: " + naiveBayes.classify(new String[]{"Anna"}).toString());
System.out.println("design: " + naiveBayes.classify(new String[]{"design"}).toString());
System.out.println("US: " + naiveBayes.classify(new String[]{"US"}).toString());

Anna: {Apple=100.0}
design: {Apple=100.0}
US: {Apple=100.0}


We now add a purchase for a Lenovo for US accounting department.

In [7]:
model.train(new String[]{"Anna", "accounting", "US"}, "Lenovo");
naiveBayes = new NaiveBayes(model);
System.out.println("Anna: " + naiveBayes.classify(new String[]{"Anna"}).toString());
System.out.println("design: " + naiveBayes.classify(new String[]{"design"}).toString());
System.out.println("US: " + naiveBayes.classify(new String[]{"US"}).toString());

Anna: {Lenovo=50.0, Apple=50.0}
design: {Lenovo=5.0E-9, Apple=50.0}
US: {Lenovo=50.0, Apple=50.0}


Nothing we couldn't figure out ourselves, yet. Anna is 50/50 as likely to buy a Lenovo or an Apple. The design department is more likely to get an Apple (50% vs. ~0%) and the US office is as likely to get a Lenovo or an Apple.

Let's now add a third user, Bill. Bill makes the same purchasing choices as Anna, but he also buys for the UK office.

In [8]:
model.train(new String[]{"Bill", "accounting", "US"}, "Lenovo");
model.train(new String[]{"Bill", "design", "US"}, "Apple");
model.train(new String[]{"Bill", "accounting", "UK"}, "Lenovo");
model.train(new String[]{"Bill", "design", "UK"}, "Apple");

In [9]:
naiveBayes = new NaiveBayes(model);
System.out.println("Anna: " + naiveBayes.classify(new String[]{"Anna"}).toString());
System.out.println("Bill: " + naiveBayes.classify(new String[]{"Anna"}).toString());
System.out.println("design: " + naiveBayes.classify(new String[]{"design"}).toString());
System.out.println("US: " + naiveBayes.classify(new String[]{"US"}).toString());

Anna: {Lenovo=16.666666666666664, Apple=16.666666666666664}
Bill: {Lenovo=16.666666666666664, Apple=16.666666666666664}
design: {Lenovo=5.0E-9, Apple=50.0}
US: {Lenovo=33.33333333333333, Apple=33.33333333333333}


Still nothing surprising. However, one of the strengths of NB is the ability to combine attributes to get insights. Let's see that adding another user, Claire. Claire will buy Lenovos for all the offices and departments.

In [10]:
model.train(new String[]{"Claire", "accounting", "US"}, "Lenovo");
model.train(new String[]{"Claire", "design", "US"}, "Lenovo");
model.train(new String[]{"Claire", "accounting", "UK"}, "Lenovo");
model.train(new String[]{"Claire", "design", "UK"}, "Lenovo");

In [11]:
naiveBayes = new NaiveBayes(model);
System.out.println("Claire: " + naiveBayes.classify(new String[]{"Claire"}).toString());
System.out.println("design: " + naiveBayes.classify(new String[]{"design"}).toString());
System.out.println("design US: " + naiveBayes.classify(new String[]{"design", "US"}).toString());
System.out.println("Bill accounting: " + naiveBayes.classify(new String[]{"Bill accounting"}).toString());

Claire: {Lenovo=40.0, Apple=3.0E-9}
design: {Lenovo=20.0, Apple=30.0}
design US: {Lenovo=11.428571428571427, Apple=20.0}
Bill accounting: {Lenovo=70.0, Apple=30.0}


Here we start to see the usefulness of combining attributes.

# Decision Trees

A classifier consisting of logical rules for traversing a tree from its root to a leaf node, with attributes of the entity modeled serving as predicates for the aforementioned logical rules. The leaf nodes then correspond to the class to which the given input is predicted.

Consider the slightly contrived example of wanting to classify a person as adult or child based solely on the attributes of height and weight. One possible decision tree for this problem is shown below:

<img src="images/BasicDecisionTree.png">

Clearly, this does not account for youth growth spurts either vertical or horizontal! However, it illustrates the architecture of a decision tree. 

With this in mind, the question of how one builds a practical decision tree classifier comes to the surface. The criteria for separating input attributes such that a class can be assigned relies on determining what logical rules, or splits, will provide the most benefit. This benefit can be pursued in a greedy fashion, wherein the best split for a given node is determined, or a long game fashion wherein an exhaustive tree is created, then pruned back to a more manageable and generalized form. 

The notion of a best split relies upon a cost function which provides a metric for assessment. For decision trees, this cost function needs to consider how many classes are included in the training data for which the split is being considered. One means for quantifying this inclusion is to use the so-called Gini Index, which provides a notion of purity of input training data to a leaf node. In this case, purity implies a smaller number of distinct classes, giving a Gini Index value of 0 when only one class is represented, 0.5 when two classes are evenly represented, and so on. The Gini Index is defined as:

$$ 
G = \sum(p_{k} * (1 – p_{k})) 
$$


where $G$ is the Gini index over all classes, and $p_{k}$ are the proportion of training instances with class $k$. 

For the generalized node case (i.e. beyond leaf nodes) the Gini score is further expanded as follows:

$$
G = \sum_{i}((1 - \sum_{j}(g_{i,j}^2))*(ng_{i}/n))
$$

where $G$ is the Gini score for the node, $i$ is iterated for all groups of contributing parent nodes, $j$ is iterated for all classes, and $g_{i,j} is the proportion of instances in group $i$ for class $j$, whilst $ng_{i}/n$ is the proportion of the training set for a given group. 


Beyond the means for assessing the quality of splits, there remains a need for knowing when to stop splitting. Naturally, splitting could continue until all training vectors are mapped to a leaf node. It will be discussed below why this is disadvantageous. Other criteria can be utilized, such as a maximum tree depth or width, or a threshold on cost function. 

## Advantages
**Intuitive** The architecture of the classifier should be apparent to anyone completing an intro data structures class. The resulting implementation is also accessible to anyone with basic programming knowledge: it's all ifs and elses! To wit: the height and weight attribute-driven example above could be coded as the following.

```java
if (this.getHeightInMeters() > 1.2) {
  this.setClassification("Adult");
}
else {
  if(this.getWeightInKilograms() > 50) {
    this.setClassification("Adult");
  }
  else {
    this.setClassification("Child");
  }
}
```


**Data flexibility** They can work with numerical and categorical data with ease; whereas other classification methods will at least require a one hot encoding of categorical variables into n binary variables where n is the number of possible values the variable can take. For example the category of department in this notebook's example would require at least two features: namely "design" and "accounting".

## Disadvantages
**Overfitting** That is, the situation arising as a result of creating a classifier which works very well for its training data but is inapplicable to input received in the wild. This can be a significant detriment to tree methods, as the training approach lends itself to human bias. 

**Heuristic-driven** While some techniques in machine learning can provide confidence on solution optimality, tree methods are not one of them. Intuition acquired via experience can help ameliorate this situation, however, in many cases grid search methods will be necessary to be certain of a tree's appropriateness. 

## Improving via ensembles
One way to improve upon the disadvantages mentioned above should be accessible to computer scientists: namely, adding a level of indirection/expansion to the classifier architecture. By utilizing an ensemble, or collection, of trees that then contribute to a final classification, a more robust classifier is made manifest. 

### Random Forests

In a random forest, a subset of the training data is selected, from which a decision tree is generated. TBC

## Example

The same minimal example as the NB case above will be used, with a Java implementation of a Decision Tree algorithm (available at https://github.com/ruivieira/java-decision-tree).


In [12]:
%maven org.ruivieira:decisiontree:0.0.2

In [13]:
import org.ruivieira.ml.decisiontree.DecisionTree;
import org.ruivieira.ml.decisiontree.TreeConfig;
import org.ruivieira.ml.decisiontree.Dataset;
import org.ruivieira.ml.decisiontree.Item;
import org.ruivieira.ml.decisiontree.features.StringValue;


import java.util.Map;

TreeConfig treeconfig = TreeConfig.create();

In [14]:
Dataset dataset = Dataset.create();
Item trainItem = Item.create();
trainItem.add("Anna", new StringValue("Apple"));
trainItem.add("design", new StringValue("Apple"));
dataset.add(trainItem);
dataset.size();
treeconfig.setData(dataset);
DecisionTree decisiontree = DecisionTree.create(treeconfig);

In [15]:
treeconfig.toString();

TreeConfig{minCount=1, decision='category', entropyThreshold=0.01, maxDepth=70, data=org.ruivieira.ml.decisiontree.Dataset@48a3bb0b, ignore=[]}

In [16]:
dataset.valueFrequency("Anna");

StringValue{data='Apple'}